In [1]:
from __future__ import print_function
import pickle
from openmoltools import openeye
import openeye.oechem as oechem
import openeye.oemedchem as oemedchem
import openeye.oedepict as oedepict
from itertools import combinations
from torsionfit.qmscan.enumfrags2pdf import main as vis_wrapper
import glob
from os.path import isfile, join
import numpy as np

In [2]:
file = open('database.pickle', 'rb')
database = pickle.load(file, encoding='latin1')
file.close()

In [3]:
to_calc = [k for k in database if (database[k]['h_conf']*4.184 > 15)]
smiles = [database[k]['smiles'] for k in to_calc]
confs = [openeye.generate_conformers(openeye.smiles_to_oemol(smiles[i]), max_confs=1) for i in range(len(to_calc))]

In [4]:
# Write out smiles and mol2 files
for i, conf in enumerate(confs):
    split = database[to_calc[i]]['nickname'].split(' ')[1:]
    fname = '{}.mol2'.format(split[0])
    isname = '{}.ism'.format(split[0])
    if len(split) > 1:
        fname = '{}.mol2'.format(split[0] + '_' + split[1])
        isname = '{}.ism'.format(split[0] + '_' + split[1])
    f = open(isname, 'w')
    f.write(smiles[i])
    f.close()
    openeye.molecule_to_mol2(conf, tripos_mol2_filename=fname)

In [5]:
smiles_files = glob.glob('/Users/chayastern/src/ChayaSt/torsionfit_examples/torsion_drive/freesolv/*.ism')

In [6]:
for infile in smiles_files:
    print(infile)
    outfile = infile.replace('ism', 'pdf')
    vis_wrapper(['enumfrags2pdf.py', '-in', infile, '-out', outfile, '-fragtype', 'ring-chain', '-min_rotors', '0'])

/Users/chayastern/src/ChayaSt/torsionfit_examples/torsion_drive/freesolv/2-nitrophenol.ism
/Users/chayastern/src/ChayaSt/torsionfit_examples/torsion_drive/freesolv/butanedioic_acid.ism
only one fragment was found
/Users/chayastern/src/ChayaSt/torsionfit_examples/torsion_drive/freesolv/d-glucose.ism
/Users/chayastern/src/ChayaSt/torsionfit_examples/torsion_drive/freesolv/diflunisal.ism
/Users/chayastern/src/ChayaSt/torsionfit_examples/torsion_drive/freesolv/flurbiprofen.ism
/Users/chayastern/src/ChayaSt/torsionfit_examples/torsion_drive/freesolv/glycerol.ism
only one fragment was found
/Users/chayastern/src/ChayaSt/torsionfit_examples/torsion_drive/freesolv/ketoprofen.ism
/Users/chayastern/src/ChayaSt/torsionfit_examples/torsion_drive/freesolv/mannitol.ism
only one fragment was found
/Users/chayastern/src/ChayaSt/torsionfit_examples/torsion_drive/freesolv/mefenamic_acid.ism
/Users/chayastern/src/ChayaSt/torsionfit_examples/torsion_drive/freesolv/xylose.ism


In [7]:
# some molecules of the set did not producea any fragements from this set. They are glycerol, mannitol and 
# butanedioic acid
for k in database:
    if 'glycerol' in database[k]['nickname']:
        glycerol = to_calc[np.where(np.asanyarray(to_calc) == k)[0][0]]
    if 'mannitol' in database[k]['nickname']:
        mannintol = to_calc[np.where(np.asanyarray(to_calc) == k)[0][0]]
    if 'butanedioic' in database[k]['nickname']:
        xylose = to_calc[np.where(np.asanyarray(to_calc) == k)[0][0]]

In [26]:
# With glycerol and mannitol, none of the fragmentation schemes seem to be working. 

In [7]:
infile = '/Users/chayastern/src/ChayaSt/torsionfit_examples/torsion_drive/freesolv/2-nitrophenol.ism'
outfile = '/Users/chayastern/src/ChayaSt/torsionfit_examples/torsion_drive/freesolv/2-nitrophenol.png'

In [8]:
# For xylose, it seems like funcgroup doesn't fragment the molecule
vis_wrapper(['enumfrags2pdf.py', '-in', infile, '-out', outfile, '-fragtype', 'ring-chain', '-min_rotors', '0'])

0

In [32]:
s = open('mannitol.ism', 'r').read()
frags = [f for f in oemedchem.OEGetBemisMurcko(openeye.smiles_to_oemol(s))]

In [34]:
mol = openeye.smiles_to_oemol(s)

In [36]:
sum([bond.IsRotor() for bond in mol.GetBonds()])

5